# 大模型

## 模型支持

### OpenAI

In [2]:
from illufly.llm import ChatOpenAI
from illufly.io import log, alog
a = ChatOpenAI(model="gpt-3.5-turbo")
log(
    a,
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    verbose=True
)

兔宝宝做梦睡，梦里长出大胡萝卜
跳来跳去嘴馋痒，咯咯笑声乐陶陶
梦醒了还在美梦中，兔宝宝开心笑哈哈
今天快乐好心情，做梦也要开心哟。



'兔宝宝做梦睡，梦里长出大胡萝卜\n跳来跳去嘴馋痒，咯咯笑声乐陶陶\n梦醒了还在美梦中，兔宝宝开心笑哈哈\n今天快乐好心情，做梦也要开心哟。'

In [3]:
a.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '兔宝宝做梦睡，梦里长出大胡萝卜\n跳来跳去嘴馋痒，咯咯笑声乐陶陶\n梦醒了还在美梦中，兔宝宝开心笑哈哈\n今天快乐好心情，做梦也要开心哟。'}]

In [4]:
await alog(
    ChatOpenAI(),
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    model="gpt-3.5-turbo",
    temperature=0.1
)

小兔子做梦梦见大胡萝卜
甜甜的梦境让它笑得合不拢嘴
梦里的世界充满了欢乐和快乐
醒来后它还在微笑，心情格外美好



'小兔子做梦梦见大胡萝卜\n甜甜的梦境让它笑得合不拢嘴\n梦里的世界充满了欢乐和快乐\n醒来后它还在微笑，心情格外美好'

**在调用时指定工具，返回应当调用的工具信息**

In [7]:
from illufly.llm import ChatOpenAI
from illufly.tools import ToolAgent
from illufly.io import log, alog
import json

def get_current_weather(location: str):
    """获取城市的天气情况"""
    yield f"{location}今天是晴天。 "
    
a = ChatOpenAI(tools=[ToolAgent(get_current_weather)])
log(a, "今天广州出门要准备什么", new_chat=True)
a.memory

广州今天是晴天。 

今天广州的天气是晴天，你出门时可以准备以下物品：

1. **防晒霜**：保护皮肤免受阳光伤害。
2. **太阳帽或太阳镜**：增加防晒效果，同时提供舒适感。
3. **水瓶**：保持水分，特别是在户外活动时。
4. **轻便的衣物**：建议穿舒适的短袖或轻薄的衣服以应对温暖的天气。
5. **鞋子**：选择舒适的鞋子，方便步行或出行。

如果你有具体的活动计划，还可以根据情况添加其他物品。希望你今天过得愉快！



[{'role': 'user', 'content': '今天广州出门要准备什么'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool', 'content': '广州今天是晴天。 '},
 {'role': 'assistant',
  'content': '今天广州的天气是晴天，你出门时可以准备以下物品：\n\n1. **防晒霜**：保护皮肤免受阳光伤害。\n2. **太阳帽或太阳镜**：增加防晒效果，同时提供舒适感。\n3. **水瓶**：保持水分，特别是在户外活动时。\n4. **轻便的衣物**：建议穿舒适的短袖或轻薄的衣服以应对温暖的天气。\n5. **鞋子**：选择舒适的鞋子，方便步行或出行。\n\n如果你有具体的活动计划，还可以根据情况添加其他物品。希望你今天过得愉快！'}]

### 智谱AI

In [9]:
from illufly.llm import ChatZhipu
from illufly.io import log, alog

zhipu = ChatZhipu()
log(
    zhipu,
    "你能帮我写一首关于兔子做梦的四句儿歌?",
    model="glm-4-flash"
)

小兔子乖乖梦飞翔，
夜幕降临入梦乡。
草丛深处歌声起，
彩虹桥上兔子跳。



'小兔子乖乖梦飞翔，\n夜幕降临入梦乡。\n草丛深处歌声起，\n彩虹桥上兔子跳。'

In [10]:
await alog(zhipu, "能改为6句？")

小兔子乖乖，梦里飞得高，
夜色温柔，轻轻把梦摇。
草地上跳，梦见自己大，
彩虹桥上，兔子跳舞笑。
星星点点，照亮兔子家，
醒来一笑，梦里花儿开。



'小兔子乖乖，梦里飞得高，\n夜色温柔，轻轻把梦摇。\n草地上跳，梦见自己大，\n彩虹桥上，兔子跳舞笑。\n星星点点，照亮兔子家，\n醒来一笑，梦里花儿开。'

In [11]:
zhipu.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '小兔子乖乖梦飞翔，\n夜幕降临入梦乡。\n草丛深处歌声起，\n彩虹桥上兔子跳。'},
 {'role': 'user', 'content': '能改为6句？'},
 {'role': 'assistant',
  'content': '小兔子乖乖，梦里飞得高，\n夜色温柔，轻轻把梦摇。\n草地上跳，梦见自己大，\n彩虹桥上，兔子跳舞笑。\n星星点点，照亮兔子家，\n醒来一笑，梦里花儿开。'}]

In [13]:
from illufly.tools import ToolAgent
from illufly.llm import ChatZhipu
from illufly.io import alog
import json

def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

await alog(
    ChatZhipu(),
    "今天广州天气如何啊",
    tools=[ToolAgent(get_current_weather)],
    model="glm-4"
)

Guangzhou今天是晴天。 

好的，我需要调用一个天气API来获取广州的天气信息。请稍等片刻今天广州天气如何啊？



'好的，我需要调用一个天气API来获取广州的天气信息。请稍等片刻今天广州天气如何啊？'

### 通义千问

In [14]:
from illufly.llm import ChatQwen
from illufly.io import log

qwen = ChatQwen()
log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

当然可以，以下是一首关于兔子做梦的儿歌：
兔子在梦中游荡，
梦见自己变成了一只小鸟，
飞上天空自由翱翔，
醒来时发现这是个美丽的梦。



'当然可以，以下是一首关于兔子做梦的儿歌：\n兔子在梦中游荡，\n梦见自己变成了一只小鸟，\n飞上天空自由翱翔，\n醒来时发现这是个美丽的梦。'

In [15]:
qwen.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以，以下是一首关于兔子做梦的儿歌：\n兔子在梦中游荡，\n梦见自己变成了一只小鸟，\n飞上天空自由翱翔，\n醒来时发现这是个美丽的梦。'}]

In [16]:
qwen.monitor_executors()

{'CHAT_OPENAI': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'DEFAULT': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'CHAT_ZHIPU': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'CHAT_QWEN': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0}}

In [18]:
from illufly.tools import ToolAgent
from illufly.llm import ChatQwen
from illufly.io import log
import json

def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

In [19]:
log(ChatQwen(), "今天广州天气如何啊", tools=[ToolAgent(get_current_weather)])
# log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

广州今天是晴天。 

广州今天的天气是晴朗的。



'广州今天的天气是晴朗的。'

## Fake

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [20]:
from illufly.llm import FakeLLM
from illufly.io import log

log(
    FakeLLM(),
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

这是一个模拟调用!



'这是一个模拟调用!'

In [21]:
FakeLLM.monitor_executors()

{'CHAT_OPENAI': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'DEFAULT': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'CHAT_ZHIPU': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0},
 'CHAT_QWEN': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0},
 'FAKE_LLM': {'max_workers': 5, 'used_workers': 0, 'waiting_threads': 0}}